In [ ]:
import geopandas as gpd

In [ ]:
water_shp = gpd.read_file("../temp/body_of_water/ne_10m_geography_marine_polys.shp", layer='ne_10m_geography_marine_polys')

In [ ]:
filter_bay = (water_shp["featurecla"] == "bay")

In [ ]:
filter_channel = (water_shp["featurecla"] == "channel")

In [ ]:
filter_fjord = (water_shp["featurecla"] == "fjord")

In [ ]:
filter_gulf = (water_shp["featurecla"] == "gulf")

In [ ]:
filter_lagoon = (water_shp["featurecla"] == "lagoon")

In [ ]:
filter_ocean = (water_shp["featurecla"] == "ocean")

In [ ]:
filter_reef = (water_shp["featurecla"] == "reef")

In [ ]:
filter_sea = (water_shp["featurecla"] == "sea")

In [ ]:
filter_sound = (water_shp["featurecla"] == "sound")

In [ ]:
filter_strait = (water_shp["featurecla"] == "strait")

In [ ]:
bay_body_of_water = water_shp.loc[filter_bay, ["name", "wikidataid", "featurecla", "geometry"]]

In [ ]:
bay_body_of_water